In [19]:
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import ELU, Input
import keras.layers as kel
from keras.optimizers import RMSprop
from keras.utils import plot_model
from keras import backend as K
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import tensorflow as tf

In [104]:
in_file = '/hepgpu3-data1/jrawling/deep_tops/csvs/ttbar.csv'
df = pd.read_csv(in_file)
print(df.head())
df.columns
# # Create the test and train data
train_columns = [ u'W_lep_eta', u'W_lep_m', u'W_lep_phi', u'W_lep_pt',
       u'jet_e_0', u'jet_e_1', u'jet_e_2', u'jet_e_3', u'jet_e_4',
       u'jet_eta_0', u'jet_eta_1', u'jet_eta_2', u'jet_eta_3', u'jet_eta_4',
       u'jet_isBjet_0', u'jet_isBjet_1', u'jet_isBjet_2', u'jet_isBjet_3',
       u'jet_isBjet_4', u'jet_phi_0', u'jet_phi_1', u'jet_phi_2', u'jet_phi_3',
       u'jet_phi_4', u'jet_pt_0', u'jet_pt_1', u'jet_pt_2', u'jet_pt_3',
       u'jet_pt_4', u'met', u'n_jets']
X = df[train_columns]

# The truth information 
truth_columns = [ u'truth_had_top_eta',
       u'truth_had_top_m', u'truth_had_top_phi', u'truth_had_top_pt',
       u'truth_lep_top_eta', u'truth_lep_top_m', u'truth_lep_top_phi',
       u'truth_lep_top_pt']
truth_columns=['truth_p0',  'truth_p1',  'truth_p2',  'truth_p3',  'truth_p4',]
Y = df[truth_columns] 

# # Split the data into test and train samples
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.3, random_state=0)

# # Padd the truth vectors
# Z_train = Y_train.copy()
# for i in xrange(5):
#     Z_train["placeholder_%d"%i] =  np.nan

# Z_train = pd.concat([X_train.copy(), Z_train],axis=1)
# Z_train = Z_train.fillna(0.0)    
# len(Z_train.columns)

   Unnamed: 0  W_lep_eta  W_lep_m  W_lep_phi  W_lep_pt  floopy_poopy  \
0           0        0.0      0.0        0.0       0.0     -1.432401   
1           1        0.0      0.0        0.0       0.0      2.300707   
2           2        0.0      0.0        0.0       0.0      3.136069   
3           3        0.0      0.0        0.0       0.0     -0.712848   
4           4        0.0      0.0        0.0       0.0      2.558781   

      jet_e_0  jet_e_1  jet_e_2  jet_e_3    ...     truth_had_top_pt  \
0  547.455750      0.0      0.0      0.0    ...           246.932047   
1  260.904984      0.0      0.0      0.0    ...           116.891859   
2  192.354422      0.0      0.0      0.0    ...           104.240453   
3  214.426625      0.0      0.0      0.0    ...            24.783420   
4  370.393562      0.0      0.0      0.0    ...           409.043125   

   truth_lep_top_eta  truth_lep_top_m  truth_lep_top_phi  truth_lep_top_pt  \
0           1.751190       172.614844           1.878099

In [101]:
output = ""
def weighted_loss(y_true, y_pred):
    # First construct the four vectors 
#     return K.mean( K.square(K.cast_to_floatx(K.argmax(y_pred[:,:5])) - K.cast_to_floatx(K.argmin(y_pred[:,:5]))),axis=-1 )
#     return K.mean( y_pred[:,K.argmax(y_pred[:,:5]) ])#  - y_true[:,K.argmax(y_pred[:,:5]) ]  ,axis=-1 )
    had_b_index = K.argmax( y_pred[0,:5] )
    lep_b_index = K.argmax( y_pred[0,:5] )
    g_index = K.argmax( y_pred[0,:5] )
    w1_index = K.argmax( y_pred[0,:5] )
    w2_index = K.argmax( y_pred[0,:5] )
    K.argm
    return K.mean( K.square(y_pred - y_true) * K.exp(-K.log(1.7) * (K.log(1. + K.exp((y_true - 3)/5 )))),axis=-1  )

def feedforward_model(input_d=4, spectator_d=10):
    """
    Creates a simple feed power NN that takes a four vector as an input and 
    returns a calibrated 4 vector 
    """
    
    first_input = Input(shape=(input_d, ))
    first_dense = Dense(35, activation='relu',)(first_input)
    
    first_dropout = Dropout(0.3)(first_dense)

    second_dense = Dense(60, activation='relu',)(first_dense)
    second_dropout = Dropout(0.3)(second_dense)
    final_layer = Dense(5, activation='softmax',)(second_dropout)
    
    model = Model(inputs=first_input, outputs=final_layer)
    if spectator_d is not None:
        spectator_in = Input(shape=(spectator_d,))
        merged_out = kel.concatenate([final_layer, spectator_in],  axis=-1)
        model = Model(inputs=[first_input, spectator_in], outputs=merged_out)
    return model 

model = feedforward_model(input_d=len(train_columns), 
                          spectator_d=None)
#                           spectator_d=len(train_columns)+len(truth_columns))
print(model.summary())

 # Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
plot_model(model,to_file='demo.eps',show_shapes=True)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_43 (InputLayer)        (None, 31)                0         
_________________________________________________________________
dense_67 (Dense)             (None, 35)                1120      
_________________________________________________________________
dense_68 (Dense)             (None, 60)                2160      
_________________________________________________________________
dropout_46 (Dropout)         (None, 60)                0         
_________________________________________________________________
dense_69 (Dense)             (None, 5)                 305       
Total params: 3,585
Trainable params: 3,585
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# Fit Model
# model.fit([X_train, pd.concat([X_train.copy(), Y_train.copy()],axis=1)], 
#           Z_train,batch_size=1,
#           epochs=1)
# Fit Model
model.fit(X_train, Y_train, epochs=50)
%time

Epoch 1/50
3046/3046 [==============================] - 1s 207us/step - loss: 133.3890 - acc: 0.1402
Epoch 2/50
3046/3046 [==============================] - 1s 199us/step - loss: 119.2509 - acc: 0.0076
Epoch 3/50
3046/3046 [==============================] - 1s 176us/step - loss: 118.5813 - acc: 0.0046
Epoch 4/50
3046/3046 [==============================] - 0s 160us/step - loss: 118.9818 - acc: 0.0108
Epoch 5/50
3046/3046 [==============================] - 0s 154us/step - loss: 119.1871 - acc: 0.0108
Epoch 6/50
3046/3046 [==============================] - 1s 174us/step - loss: 119.1660 - acc: 0.0148
Epoch 7/50
3046/3046 [==============================] - 1s 173us/step - loss: 118.9372 - acc: 0.0089
Epoch 8/50
3046/3046 [==============================] - 1s 195us/step - loss: 118.6368 - acc: 0.0026
Epoch 9/50
3046/3046 [==============================] - 1s 202us/step - loss: 118.4000 - acc: 0.0033
Epoch 10/50
3046/3046 [==============================] - 1s 193us/step - loss: 118.4748 - a

In [18]:
# evaluate the model
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))

# Save the processed variable
Y_test_pred = model.predict(X_test)
Y_train_pred = model.predict(X_train)

[-8. -8. -8. -8. -8. -8. -8. -8.]
[-9.55998003e-01  1.71792484e+02  2.32600093e-01  5.00658437e+01
 -8.62488627e-01  1.70507000e+02 -9.08043161e-02  5.54837617e+01]


In [28]:
output

''

TypeError: 'int' object is not iterable

In [36]:
import keras.layers as kel

TypeError: 'int' object is not iterable

In [114]:
# # Create the test and train data
train_columns = [ u'W_lep_eta', u'W_lep_m', u'W_lep_phi', u'W_lep_pt',
       u'jet_e_0', u'jet_e_1', u'jet_e_2', u'jet_e_3', u'jet_e_4',
       u'jet_eta_0', u'jet_eta_1', u'jet_eta_2', u'jet_eta_3', u'jet_eta_4',
       u'jet_isBjet_0', u'jet_isBjet_1', u'jet_isBjet_2', u'jet_isBjet_3',
       u'jet_isBjet_4', u'jet_phi_0', u'jet_phi_1', u'jet_phi_2', u'jet_phi_3',
       u'jet_phi_4', u'jet_pt_0', u'jet_pt_1', u'jet_pt_2', u'jet_pt_3',
       u'jet_pt_4', u'met', u'n_jets', u'truth_had_top_eta']
X = df[train_columns]

# We need the original inputs as spectator variables, so append these to our inputs 
X_copy = X.copy()
X = pd.merge(X, X_copy)

In [117]:
X.columns

Index([u'W_lep_eta', u'W_lep_m', u'W_lep_phi', u'W_lep_pt', u'jet_e_0',
       u'jet_e_1', u'jet_e_2', u'jet_e_3', u'jet_e_4', u'jet_eta_0',
       u'jet_eta_1', u'jet_eta_2', u'jet_eta_3', u'jet_eta_4', u'jet_isBjet_0',
       u'jet_isBjet_1', u'jet_isBjet_2', u'jet_isBjet_3', u'jet_isBjet_4',
       u'jet_phi_0', u'jet_phi_1', u'jet_phi_2', u'jet_phi_3', u'jet_phi_4',
       u'jet_pt_0', u'jet_pt_1', u'jet_pt_2', u'jet_pt_3', u'jet_pt_4', u'met',
       u'n_jets', u'truth_had_top_eta'],
      dtype='object')

In [122]:
import copy 
X_train = X_train + copy.copy(X_train) 

In [124]:
X_train.shape()

TypeError: 'tuple' object is not callable

In [139]:
foo =  pd.concat([X_train.copy(), Y_train.copy()], axis=1)

In [147]:
len(train_columns) + len(truth_columns)

39

In [32]:
Z_train.shape

(3047, 44)

In [80]:
z_train_val = Z_train.values

In [56]:
bar = np.argmax(z_train_val[:,:5],axis=1)
foo = z_train_val[:,bar]

In [47]:
len(foo)

3047

In [58]:
foo.shape

(3047, 3047)

In [61]:
np.min(bar)

4

In [81]:
foo = z_train_val[:,4]

array([289.8258125 , 392.149625  , 182.31004688, ..., 225.81175   ,
       471.8049375 ,  82.58546094])

In [98]:
e = z_train_val[0]
bar =  np.argmax(e[:5])
print(bar)
e[bar]

4


289.8258125

4